In [17]:
# --- Attempt 1: Check OSD-679 File List via API ---
import requests
import json # For pretty printing the output

osd_id = '679' # The dataset we are interested in
data_file_api_url = f"https://osdr.nasa.gov/osdr/data/osd/files/{osd_id}/"

print(f"Requesting file list for OSD-{osd_id} from: {data_file_api_url}")
try:
    response = requests.get(data_file_api_url)
    response.raise_for_status() # Raise an error for bad responses (4xx or 5xx)
    file_data = response.json()
    print("Successfully received API response.")

    # Let's look for files categorized as environmental or telemetry
    env_files_found = []
    if file_data.get('success') and f"OSD-{osd_id}" in file_data.get('studies', {}):
        all_files = file_data['studies'][f"OSD-{osd_id}"].get('study_files', [])
        print(f"Found {len(all_files)} total file entries listed for OSD-{osd_id} via API.")
        
        for file_info in all_files:
            filename = file_info.get('file_name', '').lower()
            category = file_info.get('category', '').lower()
            subcategory = file_info.get('subcategory', '').lower()
            
            # Keywords to look for (adjust as needed)
            env_keywords = ['environment', 'telemetry', 'temp', 'humidity', 'co2', 'radiation', 'dose']
            
            if any(keyword in category for keyword in env_keywords) or \
               any(keyword in subcategory for keyword in env_keywords) or \
               any(keyword in filename for keyword in env_keywords):
                 env_files_found.append(file_info)

    if env_files_found:
        print("\n>>> Potential Environmental Data Files Found via Study API:")
        for f_info in env_files_found:
            print(f"  - Name: {f_info.get('file_name')}")
            print(f"    Category: {f_info.get('category')}")
            print(f"    SubCategory: {f_info.get('subcategory')}")
            # To get the download URL, prepend 'https://osdr.nasa.gov' to the remote_url
            # print(f"    Download URL part: {f_info.get('remote_url')}")
    else:
        print("\n>>> No obvious environmental data files found directly under OSD-679 via the files API.")
        print(">>> Next step might be to try the Mission (RR-1) or Payload API endpoints.")

except requests.exceptions.RequestException as e:
    print(f"ERROR calling OSDR Data File API: {e}")
except json.JSONDecodeError:
    print("ERROR: Could not decode JSON response from API. Check URL or API status.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Requesting file list for OSD-679 from: https://osdr.nasa.gov/osdr/data/osd/files/679/
Successfully received API response.
Found 20 total file entries listed for OSD-679 via API.

>>> No obvious environmental data files found directly under OSD-679 via the files API.
>>> Next step might be to try the Mission (RR-1) or Payload API endpoints.


In [26]:
# --- Attempt 2: Check RR-1 Mission API Endpoint ---
import requests
import json  # To help print the results nicely

# We know the mission from OSD-679 info was RR-1
mission_id = 'RR-1'
# Construct the URL for the specific mission endpoint
mission_api_url = f"https://osdr.nasa.gov/geode-py/ws/api/mission/{mission_id}"

print(f"Requesting data for Mission '{mission_id}' from: {mission_api_url}")
try:
    response = requests.get(mission_api_url)
    response.raise_for_status()  # Check for HTTP errors
    mission_data = response.json()
    print("Successfully received API response for mission.")

    # --- Explore the structure of the response ---
    print("\nTop-level keys in mission data:", list(mission_data.keys()))

    # Does the mission data itself list associated files?
    if 'files' in mission_data and mission_data['files']:
        print("\nFound a 'files' list directly in the mission data:")
        mission_env_files = []
        # Search within these files for environmental keywords
        for file_info in mission_data['files']:
            filename = file_info.get('file_name', '').lower()
            category = file_info.get('category', '').lower()
            subcategory = file_info.get('subcategory', '').lower()
            description = file_info.get('description', '').lower()
            env_keywords = ['environment', 'telemetry', 'temp', 'humidity', 'co2', 'radiation', 'dose']

            is_env_file = any(keyword in field for keyword in env_keywords for field in [category, subcategory, filename, description] if field)

            if is_env_file:
                mission_env_files.append(file_info)

        if mission_env_files:
            print("\n>>> Potential Environmental Data Files Found via Mission API 'files' list:")
            for f_info in mission_env_files:
                print(f"  - Name: {f_info.get('file_name', 'N/A')}")
        else:
            print("\nNo obvious environmental files found in the direct 'files' list of the mission data.")
    else:
        print("\nNo direct 'files' list found at the top level of the mission data.")

    # Check if the mission links to Payloads or other related items
    mission_parents = mission_data.get('parents', {})
    if mission_parents and isinstance(mission_parents, dict):
        print("\nMission data contains links to related items (Payloads, Experiments, Studies):")
        print(json.dumps(mission_parents, indent=2))
        print("\n>>> Environmental files might be associated with linked Payloads (like Rodent Habitat). We may need to query the Payload API next using identifiers found here.")
    else:
        print("\nNo 'parents' structure found linking to Payloads etc.")

except requests.exceptions.RequestException as e:
    print(f"ERROR calling OSDR Mission API: {e}")
except json.JSONDecodeError:
    print("ERROR: Could not decode JSON response from API. Check URL or API status for Mission endpoint.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Requesting data for Mission 'RR-1' from: https://osdr.nasa.gov/geode-py/ws/api/mission/RR-1
ERROR calling OSDR Mission API: 400 Client Error: BAD REQUEST for url: https://osdr.nasa.gov/geode-py/ws/api/mission/RR-1


In [27]:
# Cell 3: Attempt 3 - List ALL Missions to find RR-1 Info
import requests
import json
from urllib.parse import urlparse # Added import just in case

all_missions_url = "https://osdr.nasa.gov/geode-py/ws/api/missions" # Plural 'missions' endpoint

print(f"Requesting list of all missions from: {all_missions_url}")
try:
    response = requests.get(all_missions_url)
    response.raise_for_status()
    missions_response_data = response.json() # Changed variable name
    print("Successfully received API response for all missions.")

    # --- Find RR-1 in the list (using the structure we discovered) ---
    rr1_found = False
    # Check if the response is a dictionary and has the 'data' key which is a list
    if isinstance(missions_response_data, dict) and 'data' in missions_response_data and isinstance(missions_response_data['data'], list):
        actual_missions_list = missions_response_data['data']
        print(f"Found {len(actual_missions_list)} total mission entries under the 'data' key.")

        for i, mission_entry in enumerate(actual_missions_list): # Iterate through the actual list
            mission_url = mission_entry.get('mission')
            potential_id = "ERROR_PARSING_URL" # Default
            if mission_url:
                try:
                    # Get the last part of the URL path after splitting by '/'
                    path_parts = urlparse(mission_url).path.split('/')
                    # Take the last non-empty part as the potential ID
                    potential_id = next((part for part in reversed(path_parts) if part), None)
                    if potential_id is None: potential_id = "PARSE_FAILED_NO_PART"
                except Exception:
                     potential_id = "ERROR_DURING_PARSE" # Handle errors during parsing
            else:
                potential_id = "MISSING_URL" # Handle entries without a URL

            # Check if this ID is 'rr-1' or 'spacex-4' (case-insensitive)
            # We will fetch details later if needed, first just identify potential matches
            if potential_id and (potential_id.lower() == 'rr-1' or potential_id.lower() == 'spacex-4'):
                 print(f"\n>>> Found potential match at Entry {i+1} with Potential ID: '{potential_id}'")
                 print(f"    Full Mission Entry from List: {mission_entry}")
                 # We found a likely candidate, store its info if needed later
                 # If we want to be sure, we'd call requests.get(mission_url).json() here
                 rr1_found = True
                 # Let's not break, maybe there are multiple matches? (e.g., RR-1 and SpaceX-4 separate)
                 # break

        if not rr1_found:
            print("\nCould not automatically find 'RR-1' or 'SpaceX-4' as the identifier in the mission URLs.")
            print("You might need to manually inspect the list of IDs printed in the previous run if available, or uncomment code below.")
            # Optional: Print the list of potential IDs again if needed
            # print("\nList of Potential IDs found:")
            # for i, mission_entry in enumerate(actual_missions_list):
            #    mission_url = mission_entry.get('mission')
            #    # ... (code to extract and print potential_id) ...

    else:
        print("API response structure was not the expected {'data': [...list...]} format.")
        # print("Printing structure again:")
        # print(json.dumps(missions_response_data, indent=2))

except Exception as e:
    print(f"An ERROR occurred: {e}")

Requesting list of all missions from: https://osdr.nasa.gov/geode-py/ws/api/missions
Successfully received API response for all missions.
Found 152 total mission entries under the 'data' key.

>>> Found potential match at Entry 92 with Potential ID: 'SpaceX-4'
    Full Mission Entry from List: {'mission': 'https://osdr.nasa.gov/geode-py/ws/api/mission/SpaceX-4'}


In [20]:
# Cell 4: Attempt 4 - Query Specific Mission using 'SpaceX-4' ID
import requests
import json
from urllib.parse import urlparse # Might need later if response contains URLs

# Use the identifier found in the previous step's output
mission_id = 'SpaceX-4' # Use the identifier we just confirmed!
specific_mission_url = f"https://osdr.nasa.gov/geode-py/ws/api/mission/{mission_id}"

print(f"Requesting details for Mission '{mission_id}' from: {specific_mission_url}")
try:
    response = requests.get(specific_mission_url)
    response.raise_for_status() # Check for HTTP errors
    mission_details = response.json()
    print(f"Successfully received API response for mission {mission_id}.")

    # --- Look for Environmental Files OR Payload Links ---
    env_files_found_in_mission = []
    payload_urls = [] # To store links to payloads

    # Check if there's a 'files' list directly within the mission details
    if 'files' in mission_details and isinstance(mission_details.get('files'), list):
        print("\nFound a 'files' list directly in the mission details:")
        file_list_to_check_mission = mission_details['files']
        for file_info in file_list_to_check_mission:
            # Extract details (lowercase for easier matching)
            filename = file_info.get('file_name', '').lower()
            category = file_info.get('category', '').lower()
            subcategory = file_info.get('subcategory', '').lower()
            description = file_info.get('description', '').lower()
            # Keywords to identify environmental data
            env_keywords = ['environment', 'telemetry', 'temp', 'humidity', 'co2', 'radiation', 'dose']
            is_env_file = any(keyword in field for keyword in env_keywords for field in [category, subcategory, filename, description] if field)
            if is_env_file:
                env_files_found_in_mission.append(file_info)
    else:
         print("\nNo direct 'files' list found in the mission details.")

    # Check if the mission links to Payloads (structure might be under 'parents' or similar)
    # Adjust the keys ('parents', 'payload') if needed based on actual JSON structure seen previously or in API docs
    if 'parents' in mission_details and isinstance(mission_details.get('parents'), dict):
        if 'payload' in mission_details['parents'] and isinstance(mission_details['parents']['payload'], list):
            print("\nMission links to the following Payloads:")
            for payload_link in mission_details['parents']['payload']:
                payload_url = payload_link.get('payload') # Get the URL value
                if payload_url:
                    print(f"  - {payload_url}")
                    payload_urls.append(payload_url)
            if not payload_urls: print("No payload URLs found in the links.")
        else: print("\nNo 'payload' list found within 'parents' structure.")
    else: print("\nNo 'parents' structure found linking to Payloads in this mission's details.") # Adjust if structure is different

    # --- Report Findings ---
    if env_files_found_in_mission:
        print("\n>>> Potential Environmental Data Files Found DIRECTLY within SpaceX-4 Mission Data:")
        for f_info in env_files_found_in_mission:
             print(f"  - Name: {f_info.get('file_name', 'N/A')}, Category: {f_info.get('category', 'N/A')}, SubCategory: {f_info.get('subcategory', 'N/A')}")
             # Add download URL info if needed: print(f"    URL Part: {f_info.get('remote_url')}")
    elif payload_urls:
        print("\n>>> Environmental files NOT found directly, but found links to Payloads.")
        print(">>> Next step is likely querying the relevant Payload URL (e.g., the one containing 'RR-1').")
    else: # If no direct files AND no payload links found
         print("\n>>> Summary: No direct environmental files or links to payloads containing them were found for the SpaceX-4 mission via this API call.")
         # Optionally print full details if we get stuck again
         # print("\n--- Full SpaceX-4 Mission Details ---")
         # print(json.dumps(mission_details, indent=2))

except requests.exceptions.RequestException as e:
    print(f"ERROR calling OSDR specific Mission API for {mission_id}: {e}")
except json.JSONDecodeError:
    print(f"ERROR: Could not decode JSON response for mission {mission_id}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Requesting details for Mission 'SpaceX-4' from: https://osdr.nasa.gov/geode-py/ws/api/mission/SpaceX-4
Successfully received API response for mission SpaceX-4.

Found a 'files' list directly in the mission details:

Mission links to the following Payloads:
  - https://osdr.nasa.gov/geode-py/ws/api/payload/Seedling%20Growth-2
  - https://osdr.nasa.gov/geode-py/ws/api/payload/RR-1
  - https://osdr.nasa.gov/geode-py/ws/api/payload/Micro-8
  - https://osdr.nasa.gov/geode-py/ws/api/payload/AFEx
  - https://osdr.nasa.gov/geode-py/ws/api/payload/RR-1_BSP
  - https://osdr.nasa.gov/geode-py/ws/api/payload/ARC00XX133
  - https://osdr.nasa.gov/geode-py/ws/api/payload/BRIC-19

>>> Environmental files NOT found directly, but found links to Payloads.
>>> Next step is likely querying the relevant Payload URL (e.g., the one containing 'RR-1').


In [21]:
# --- Updated Cell 5: Loop Through Linked Payloads from RR-1 Mission ---
import requests
import json

# Paste in the 'parents' data from Cell 2 here if not shared across cells
# Ideally, you'd store it from Cell 2 like: mission_parents = mission_data.get('parents', {})
# But for standalone use, you can copy that structure manually if needed

# For now, assuming you saved this earlier:
# mission_parents = mission_data.get('parents', {}) ← from Cell 2

# Replace this with the correct mission_parents if needed:
if 'mission_data' not in globals():
    print("Please run Cell 2 first so 'mission_data' is defined.")
else:
    mission_parents = mission_data.get('parents', {})
    payload_ids = mission_parents.get('payload', [])

    if not payload_ids:
        print("No linked payloads found in mission_data['parents'].")
    else:
        print(f"\nFound {len(payload_ids)} linked payloads in mission_data['parents']['payload']")

        env_keywords = ['environment', 'telemetry', 'temp', 'humidity', 'co2', 'radiation', 'dose']

        for payload_id in payload_ids:
            print(f"\n--- Checking Payload ID: {payload_id} ---")
            payload_api_url = f"https://osdr.nasa.gov/geode-py/ws/api/payload/{payload_id}"
            try:
                response = requests.get(payload_api_url)
                response.raise_for_status()
                payload_data = response.json()

                if 'files' in payload_data and payload_data['files']:
                    found_env_files = []
                    for file_info in payload_data['files']:
                        fields_to_check = [
                            file_info.get('file_name', '').lower(),
                            file_info.get('category', '').lower(),
                            file_info.get('subcategory', '').lower(),
                            file_info.get('description', '').lower()
                        ]
                        if any(keyword in field for keyword in env_keywords for field in fields_to_check if field):
                            found_env_files.append(file_info)

                    if found_env_files:
                        print(f"  >>> Found {len(found_env_files)} environmental-related files:")
                        for f in found_env_files:
                            print(f"    - {f.get('file_name')} | {f.get('remote_url', 'No URL')}")
                    else:
                        print("  No environmental-related files found in this payload.")
                else:
                    print("  No 'files' list in this payload.")

            except Exception as e:
                print(f"  ERROR retrieving payload {payload_id}: {e}")


Please run Cell 2 first so 'mission_data' is defined.


In [25]:
# Attempt to fetch the list of missions
import requests

mission_list_url = 'https://osdr.nasa.gov/geode-py/ws/api/mission'
print(f"Fetching mission list from {mission_list_url}")

try:
    response = requests.get(mission_list_url)
    response.raise_for_status()  # Check for HTTP errors
    mission_list = response.json()
    print("Successfully retrieved mission list.")
    print(mission_list)  # Check what mission list data we get

except requests.exceptions.RequestException as e:
    print(f"Error retrieving mission list: {e}")



Fetching mission list from https://osdr.nasa.gov/geode-py/ws/api/mission
Error retrieving mission list: 404 Client Error: NOT FOUND for url: https://osdr.nasa.gov/geode-py/ws/api/mission


In [28]:
import requests
import json

# Mission ID for SpaceX-4
mission_id = 'SpaceX-4'
# Construct the URL for the specific mission endpoint
mission_api_url = f"https://osdr.nasa.gov/geode-py/ws/api/mission/{mission_id}"

print(f"Requesting data for Mission '{mission_id}' from: {mission_api_url}")
try:
    response = requests.get(mission_api_url)
    response.raise_for_status()  # Check for HTTP errors
    mission_data = response.json()
    print("Successfully received API response for mission.")

    # --- Explore the structure of the response ---
    # Let's see what top-level information is available for the mission
    print("\nTop-level keys in mission data:", list(mission_data.keys()))

    # Does the mission data itself list associated files?
    if 'files' in mission_data and mission_data['files']:
        print("\nFound a 'files' list directly in the mission data:")
        mission_env_files = []
        # Search within these files for environmental keywords
        for file_info in mission_data['files']:
            filename = file_info.get('file_name', '').lower()
            category = file_info.get('category', '').lower()
            subcategory = file_info.get('subcategory', '').lower()
            description = file_info.get('description', '').lower()
            env_keywords = ['environment', 'telemetry', 'temp', 'humidity', 'co2', 'radiation', 'dose']

            # Check if keywords appear in relevant fields
            is_env_file = any(keyword in field for keyword in env_keywords for field in [category, subcategory, filename, description] if field)

            if is_env_file:
                mission_env_files.append(file_info)

        if mission_env_files:
            print("\n>>> Potential Environmental Data Files Found via Mission API 'files' list:")
            for f_info in mission_env_files:
                print(f"  - Name: {f_info.get('file_name', 'N/A')}")
                # You can add more details here if needed, like description or category
        else:
            print("\nNo obvious environmental files found in the direct 'files' list of the mission data.")
    else:
        print("\nNo direct 'files' list found at the top level of the mission data.")

    # Check if the mission links to Payloads or other related items where files might be
    if 'parents' in mission_data and isinstance(mission_data.get('parents'), dict):
        print("\nMission data contains links to related items (Payloads, Experiments, Studies):")
        # Pretty print the linked items section
        print(json.dumps(mission_data['parents'], indent=2))
        print("\n>>> Environmental files might be associated with linked Payloads (like Rodent Habitat). We may need to query the Payload API next using identifiers found here.")
    else:
        print("\nNo 'parents' structure found linking to Payloads etc.")

except requests.exceptions.RequestException as e:
    print(f"ERROR calling OSDR Mission API: {e}")
except json.JSONDecodeError:
    print("ERROR: Could not decode JSON response from API. Check URL or API status for Mission endpoint.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Requesting data for Mission 'SpaceX-4' from: https://osdr.nasa.gov/geode-py/ws/api/mission/SpaceX-4
Successfully received API response for mission.

Top-level keys in mission data: ['id', 'identifier', 'identifierLowercase', 'esID', 'aliases', 'startDate', 'endDate', 'files', 'vehicle', 'people', 'versionInfo', 'parents']

No direct 'files' list found at the top level of the mission data.

Mission data contains links to related items (Payloads, Experiments, Studies):
{
  "payload": [
    {
      "payload": "https://osdr.nasa.gov/geode-py/ws/api/payload/Seedling%20Growth-2"
    },
    {
      "payload": "https://osdr.nasa.gov/geode-py/ws/api/payload/RR-1"
    },
    {
      "payload": "https://osdr.nasa.gov/geode-py/ws/api/payload/Micro-8"
    },
    {
      "payload": "https://osdr.nasa.gov/geode-py/ws/api/payload/AFEx"
    },
    {
      "payload": "https://osdr.nasa.gov/geode-py/ws/api/payload/RR-1_BSP"
    },
    {
      "payload": "https://osdr.nasa.gov/geode-py/ws/api/payload/ARC0

In [29]:
import requests

# List of payload URLs from the SpaceX-4 mission
payload_urls = [
    "https://osdr.nasa.gov/geode-py/ws/api/payload/Seedling%20Growth-2",
    "https://osdr.nasa.gov/geode-py/ws/api/payload/RR-1",
    "https://osdr.nasa.gov/geode-py/ws/api/payload/Micro-8",
    "https://osdr.nasa.gov/geode-py/ws/api/payload/AFEx",
    "https://osdr.nasa.gov/geode-py/ws/api/payload/RR-1_BSP",
    "https://osdr.nasa.gov/geode-py/ws/api/payload/ARC00XX133",
    "https://osdr.nasa.gov/geode-py/ws/api/payload/BRIC-19"
]

for url in payload_urls:
    try:
        print(f"\nFetching payload data from: {url}")
        resp = requests.get(url)
        resp.raise_for_status()
        data = resp.json()
        print(f"Payload ID: {data.get('identifier', 'N/A')}")
        if 'files' in data and data['files']:
            for f in data['files']:
                print(f"  - File: {f.get('fileName', 'Unnamed')} ({f.get('type', 'Unknown')})")
        else:
            print("  No files listed in this payload.")
    except Exception as e:
        print(f"  Error fetching payload: {e}")



Fetching payload data from: https://osdr.nasa.gov/geode-py/ws/api/payload/Seedling%20Growth-2
Payload ID: Seedling Growth-2
  No files listed in this payload.

Fetching payload data from: https://osdr.nasa.gov/geode-py/ws/api/payload/RR-1
Payload ID: RR-1
  - File: Unnamed (Unknown)
  - File: Unnamed (Unknown)
  - File: Unnamed (Unknown)
  - File: Unnamed (Unknown)
  - File: Unnamed (Unknown)
  - File: Unnamed (Unknown)
  - File: Unnamed (Unknown)
  - File: Unnamed (Unknown)
  - File: Unnamed (Unknown)
  - File: Unnamed (Unknown)

Fetching payload data from: https://osdr.nasa.gov/geode-py/ws/api/payload/Micro-8
Payload ID: Micro-8
  No files listed in this payload.

Fetching payload data from: https://osdr.nasa.gov/geode-py/ws/api/payload/AFEx
Payload ID: AFEx
  No files listed in this payload.

Fetching payload data from: https://osdr.nasa.gov/geode-py/ws/api/payload/RR-1_BSP
Payload ID: RR-1_BSP
  No files listed in this payload.

Fetching payload data from: https://osdr.nasa.gov/ge

In [30]:
import requests
import pandas as pd

# List of OSD IDs to query
osd_ids = ['OSD-100', 'OSD-162', 'OSD-557']  # Add more as needed

all_records = []

for osd_id in osd_ids:
    print(f"🔍 Fetching environmental data for {osd_id}...")
    api_url = f'https://osdr.nasa.gov/geode-py/ws/api/environmental/osd_id={osd_id}'
    response = requests.get(api_url)
    
    if response.status_code == 200:
        data = response.json()
        for entry in data.get("environmentalRecords", []):
            record = {
                "OSD_ID": osd_id,
                "MeasurementType": entry.get("type", "Unknown"),
                "Value": entry.get("value"),
                "Units": entry.get("units"),
                "Timestamp": entry.get("timestamp"),
                "SPOKE_Node": "Measurement"  # Placeholder for mapping
            }
            all_records.append(record)
    else:
        print(f"⚠️ Failed to fetch data for {osd_id} ({response.status_code})")

# Combine all data into a single DataFrame
df_env = pd.DataFrame(all_records)
print(f"✅ Retrieved {len(df_env)} environmental records across {len(osd_ids)} OSDs")
df_env.head()


🔍 Fetching environmental data for OSD-100...
⚠️ Failed to fetch data for OSD-100 (403)
🔍 Fetching environmental data for OSD-162...
⚠️ Failed to fetch data for OSD-162 (403)
🔍 Fetching environmental data for OSD-557...
⚠️ Failed to fetch data for OSD-557 (403)
✅ Retrieved 0 environmental records across 3 OSDs


""


In [31]:
import requests

# Function to fetch assay data in JSON format for a specific OSD
def fetch_assay_data(osd_id):
    url = f"https://visualization.osdr.nasa.gov/biodata/api/v2/dataset/{osd_id}/assays/?format=json"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json()  # Return the JSON response
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {osd_id}: {e}")
        return None

# Example OSD IDs to fetch data for
osd_ids = ["OSD-557", "OSD-100", "OSD-162"]

# Fetch and print assay data for each OSD ID
for osd_id in osd_ids:
    print(f"Fetching assay data for {osd_id}...")
    assay_data = fetch_assay_data(osd_id)
    if assay_data:
        print(f"Assay data for {osd_id}:")
        print(assay_data)
    else:
        print(f"Failed to fetch assay data for {osd_id}")


Fetching assay data for OSD-557...
Assay data for OSD-557:
{'OSD-557': {'assays': {'OSD-557_bone-microstructure_micro-computed-tomography_SkyScan_1272_desktop_micro-CT_system': {'REST_URL': 'https://visualization.osdr.nasa.gov/biodata/api/v2/dataset/OSD-557/assay/OSD-557_bone-microstructure_micro-computed-tomography_SkyScan_1272_desktop_micro-CT_system/'}, 'OSD-557_histological-assay_light-microscopy_Inverted fluorescence microscope': {'REST_URL': 'https://visualization.osdr.nasa.gov/biodata/api/v2/dataset/OSD-557/assay/OSD-557_histological-assay_light-microscopy_Inverted%20fluorescence%20microscope/'}, 'OSD-557_molecular-cellular-imaging_immunostaining_4-HNE': {'REST_URL': 'https://visualization.osdr.nasa.gov/biodata/api/v2/dataset/OSD-557/assay/OSD-557_molecular-cellular-imaging_immunostaining_4-HNE/'}, 'OSD-557_molecular-cellular-imaging_immunostaining_PNA': {'REST_URL': 'https://visualization.osdr.nasa.gov/biodata/api/v2/dataset/OSD-557/assay/OSD-557_molecular-cellular-imaging_immu